<a href="https://colab.research.google.com/github/SamarthJ03/Quora-Question-pair-similarity/blob/main/questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import  re
from textblob import TextBlob
from wordcloud import WordCloud
import keras


In [ ]:
df = pd.read_csv("/content/train.csv")

In [ ]:
df.drop(columns='id',inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404290 non-null  int64 
 1   qid2          404290 non-null  int64 
 2   question1     404289 non-null  object
 3   question2     404288 non-null  object
 4   is_duplicate  404290 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 15.4+ MB


In [ ]:
df.isnull().sum()

qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [ ]:
print(df[df['question2'].isnull()==True])
print(df[df['question1'].isnull()==True])

          qid1    qid2                         question1 question2  \
105780  174363  174364    How can I develop android app?       NaN   
201841  303951  174364  How can I create an Android app?       NaN   

        is_duplicate  
105780             0  
201841             0  
          qid1    qid2 question1  \
363362  493340  493341       NaN   

                                                question2  is_duplicate  
363362  My Chinese name is Haichao Yu. What English na...             0  


In [ ]:
df_t = df.copy()

In [ ]:
df_t.dropna(inplace=True)

In [ ]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404287 entries, 0 to 404289
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   qid1          404287 non-null  int64 
 1   qid2          404287 non-null  int64 
 2   question1     404287 non-null  object
 3   question2     404287 non-null  object
 4   is_duplicate  404287 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 18.5+ MB


In [ ]:
df_t.describe(include='all')

,qid1,qid2,question1,question2,is_duplicate
count,404287.000000,404287.000000,404287,404287,404287.000000
unique,NaN,NaN,290455,299173,NaN
top,NaN,NaN,How do I improve my English speaking?,How can you look at someone's private Instagra...,NaN
freq,NaN,NaN,50,120,NaN
mean,217243.151093,220955.212082,NaN,NaN,0.369201
std,157751.614317,159903.168488,NaN,NaN,0.482589
min,1.000000,2.000000,NaN,NaN,0.000000
25%,74436.500000,74726.500000,NaN,NaN,0.000000
50%,192181.000000,197053.000000,NaN,NaN,0.000000
75%,346573.000000,354692.000000,NaN,NaN,1.000000


In [ ]:
df_t.duplicated().sum()

0

In [ ]:
df_t['questions'] = df_t['question1']+df_t["question2"]

In [ ]:
df_t['questions'].duplicated().sum()

0

In [ ]:
df_t.drop(columns='questions',inplace=True)

In [ ]:
df_t.reset_index(inplace=True)

In [ ]:
df_t.drop(columns='index',inplace=True)

In [ ]:
x = df_t.drop(columns=['qid1','qid2','is_duplicate'])

In [ ]:
y = df_t['is_duplicate']

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:

from nltk.corpus import stopwords

sw_list = stopwords.words('english')
x['question1'] = x['question1'].apply(lambda y: [item for item in y.split() if item not in sw_list]).apply(lambda y:" ".join(y))
import string
exclude = string.punctuation
def remove_punc1(text):
    return text.translate(str.maketrans('', '', exclude))
x['question1'] = x['question1'].apply(remove_punc1)

In [ ]:
x['question2'] = x['question2'].apply(lambda y: [item for item in y.split() if item not in sw_list]).apply(lambda y:" ".join(y))
x['question2'] = x['question2'].apply(remove_punc1)

In [ ]:
x

,question1,question2
0,What step step guide invest share market india,What step step guide invest share market
1,What story Kohinoor KohiNoor Diamond,What would happen Indian government stole Kohi...
2,How I increase speed internet connection using...,How Internet speed increased hacking DNS
3,Why I mentally lonely How I solve it,Find remainder math2324math divided 2423
4,Which one dissolve water quikly sugar salt met...,Which fish would survive salt water
...,...,...
404282,How many keywords Racket programming language ...,How many keywords PERL Programming Language la...
404283,Do believe life death,Is true life death
404284,What one coin,Whats coin
404285,What approx annual cost living studying UIC Ch...,I little hairfall problem I want use hair styl...


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
x['question1'] = x['question1'].apply(lambda x: wordnet_lemmatizer.lemmatize(x) ).apply(lambda x:"".join(x))
x['question2'] = x['question2'].apply(lambda x: wordnet_lemmatizer.lemmatize(x) ).apply(lambda x:"".join(x))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(oov_token='<nothing>')

In [ ]:
sentences = x['question1']+ " " + x['question2']

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
len(tokenizer.word_index)

111669

In [ ]:
train_df = tokenizer.texts_to_sequences(sentences)


In [ ]:
maxi = 0;
for i in range(len(train_df)):
  maxi = max(len(train_df[i]),maxi)
# train_df = pad_sequences(train_df, maxlen=maxi, padding='post')

In [ ]:
maxi

147

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(train_df,y,test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
import tensorflow as tf

In [ ]:
from keras.layers import Embedding

In [ ]:
model = Sequential()
model.add(Embedding(111669,2,input_length=147))
model.add(Bidirectional((LSTM(32,return_sequences=True))))
model.add(Bidirectional((LSTM(32,return_sequences=True))))
model.add(Bidirectional((LSTM(32,return_sequences=False))))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test), callbacks=[early_stopping])


In [ ]:
model.save('my_model.h5')

In [ ]:
from google.colab import files

# Download the model file
files.download('my_model.h5')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [97]:
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")

<ipython-input-97-bca5cd1fcd97>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/content/drive/MyDrive/test.csv")


In [98]:
test_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
3563470,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
3563471,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
3563472,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
3563473,2345794,What were the best and worst things about publ...,What are the best and worst things examination...


In [ ]:
test_df.isnull().sum()

test_id      0
question1    4
question2    6
dtype: int64

In [ ]:
test_df.describe(include='all')

,test_id,question1,question2
count,3563475,3563471,3563469
unique,2607940,2211008,2227399
top,1303970,What,What
freq,2,2033,2016


In [ ]:
test_df[test_df['question1'].isnull()]

,test_id,question1,question2
1046690,1046690,NaN,How I what can learn android app development?
1461432,1461432,NaN,How distinct can learn android app development?
2345796,"life in dublin?""",NaN,NaN
2679111,1461432,NaN,How distinct can learn android app development?


In [ ]:
test_df['question1'].fillna('What',inplace=True)
test_df['question2'].fillna('What',inplace=True)


In [ ]:
test_df['question1'] = test_df['question1'].apply(lambda y: [item for item in y.split() if item not in sw_list]).apply(lambda y:" ".join(y))
test_df['question1'] = test_df['question1'].apply(remove_punc1)
test_df['question2'] = test_df['question2'].apply(lambda y: [item for item in y.split() if item not in sw_list]).apply(lambda y:" ".join(y))
test_df['question2'] = test_df['question2'].apply(remove_punc1)
test_df['question1'] = test_df['question1'].apply(lambda x: wordnet_lemmatizer.lemmatize(x) ).apply(lambda x:"".join(x))
test_df['question2'] = test_df['question2'].apply(lambda x: wordnet_lemmatizer.lemmatize(x) ).apply(lambda x:"".join(x))

In [ ]:
tests = test_df['question1']+ " " + test_df['question2']
test = tokenizer.texts_to_sequences(tests)


In [ ]:
test = pad_sequences(test, maxlen=maxi, padding='post')

In [ ]:
model = tf.keras.models.load_model('/content/my_model.h5')

# Check the model summary to ensure it is loaded correctly
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 147, 2)            223338    
                                                                 
 bidirectional_8 (Bidirecti  (None, 147, 64)           8960      
 onal)                                                           
                                                                 
 bidirectional_9 (Bidirecti  (None, 147, 64)           24832     
 onal)                                                           
                                                                 
 bidirectional_10 (Bidirect  (None, 64)                24832     
 ional)                                                          
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                      

In [94]:
predictions = model.predict(test)

111359/111359 [==============================] - 8844s 79ms/step


In [101]:
predictions

array([[0.03249224],
       [0.32331738],
       [0.3084731 ],
       ...,
       [0.00485544],
       [0.01650714],
       [0.9153995 ]], dtype=float32)